In [ ]:
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch
import re
from torch.utils.data import DataLoader

c:\Users\user1\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 모델 및 학습 관련 파라미터 설정

In [ ]:
model_parameters = {
    'time' :30, # 각 노래에서 몇 초를 가져올 것인지
    'sample_rate' :44100 # [1, sample_rate*time]: time(초)로 구간 설정    
}

learning_parameters = {
    'dataset_dir':'mp3',
    'device' : torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'epoch': 20,
    'batch_size' : 64,
    'lr' : 1e-4,
    'lr_decay' :0.95,
    'ckpt_dir' : None #학습중인 모델의 경로로
}

### 데이터로더

In [ ]:
from data import create_contrastive_datasets, create_datsets, ContrastiveDataset

# 오디오 파일 경로 및 데이터셋 준비
train_dataset = create_datsets(dataset_dir= learning_parameters['dataset_dir'],
                               state = 'train')

# ContrastiveDataset으로 변환
train_contrastive_dataset = ContrastiveDataset(train_dataset, model_parameters)

# DataLoader로 배치 생성
train_loader = DataLoader(train_contrastive_dataset, 
                          batch_size=learning_parameters['batch_size'], 
                          shuffle=True, 
                          drop_last =True) 
#-> 한 배치의 구성 : clip_a, clip_b, file_id

# 데이터로더 확인용
print('train dataset 크기')
print(train_contrastive_dataset.__len__())

### 모델

In [ ]:
from models import ContrastiveModel
from ast_encoder import ASTEncoder
from loss import soft_info_nce_loss, info_nce_loss
from loss_weight import generate_lyrics_embeddings, compute_similarity
from transformers import BertTokenizer, BertModel


# 1. 모델과 옵티마이저 초기화

ast_encoder = ASTEncoder()
ast_encoder.set_train_mode()
ast_encoder.to(learning_parameters['device'])


model = ContrastiveModel(ast_encoder)

# 체크포인트 불러오기
if learning_parameters['ckpt_dir']:
    checkpoint = torch.load(learning_parameters['ckpt_dir'], map_location=learning_parameters['device'])
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Checkpoint loaded from {learning_parameters['ckpt_dir']}")

model.to(learning_parameters['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

#scheduler 추가
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer=optimizer,
    lr_lambda=lambda epoch: learning_parameters['lr_decay'] ** epoch,
    last_epoch=-1,
    verbose=False
    )

# 2. BERT 모델 로드 (가사 임베딩용)
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model.to(learning_parameters['device'])
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

### 학습

In [ ]:
import pytorch_lightning as pl
from train import AudioLyricsModel

# AudioLyricsModel 인스턴스 생성
audio_lyrics_model = AudioLyricsModel(
    model=model,
    lyrics=True,  # 가사 사용 여부 설정 (필요에 따라 True/False 변경)
    bert_model=bert_model,
    tokenizer=tokenizer,
    batch_size=learning_parameters['batch_size']
)

trainer = pl.Trainer(max_epochs=learning_parameters['epoch'])
trainer.fit(audio_lyrics_model, train_loader)